## Imports, Settings, & Functions

In [1]:
import cv2
import PIL.Image as Image
import os
from scipy.io import loadmat
from scipy.io.matlab import mat_struct
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import json
from tqdm import tqdm
from collections import defaultdict

In [2]:
root_dir = "/home/linux-pc/gh/projects/NeuralNexus/New-Features/Thought-to-Image/V1-Visual-Cortex-Visualization"
data_root_dir = root_dir + "/data/crcns-pvc1/crcns-ringach-data/"
movie_root_dir = "movie_frames/movie000_000.images/"
current_movie = "movie000_000_000.jpeg"
neuro_root_dir = "neurodata/ac1/"
current_neuro_data = "ac1_u004_000.mat"

In [3]:
movie_frame_path = '/home/linux-pc/gh/projects/NeuralNexus/New-Features/Thought-to-Image/V1-Visual-Cortex-Visualization/data/crcns-pvc1/crcns-ringach-data/movie_frames/movie000_000.images/movie000_000_000.jpeg'

In [4]:
def matlab_to_python(obj):
    """Recursively convert MATLAB structs and cell arrays to Python dicts/lists."""
    if isinstance(obj, mat_struct):
        return {field: matlab_to_python(getattr(obj, field)) for field in obj._fieldnames}
    elif isinstance(obj, np.ndarray):
        # Handle MATLAB cell arrays
        if obj.dtype == object:
            return [matlab_to_python(el) for el in obj.flat]
        else:
            return obj
    elif isinstance(obj, list):
        return [matlab_to_python(el) for el in obj]
    else:
        return obj


In [5]:
def analyze_waveform(pepANA_clean, condition=0, electrode=0, verbose=False):
    """
    Analyze SVD of waveforms from a specific condition and electrode in pepANA.

    Parameters:
        pepANA_clean: dict - Unpacked MATLAB structure from loadmat
        condition: int - Index of the condition (0-based)
        electrode: int - Index of the electrode (0-based)
        verbose: bool - Whether to show plots

    Returns:
        signal_idx: np.ndarray - Indices of waveforms classified as signal
    """

    # Extract waveform matrix [time x waveforms]
    waveforms = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1]
    waveforms = np.array(waveforms, dtype=np.float64)  # Ensure float precision

    # Perform SVD
    u, s, vT = np.linalg.svd(waveforms, full_matrices=False)  # vT: shape [n_waveforms, n_waveforms]

    # Projection into first 2 principal components
    projections = vT.T[:, :2]  # shape: [n_waveforms, 2]

    # Plot SVD projections
    if verbose:
        plt.figure(figsize=(8, 6))
        plt.scatter(projections[:, 0], projections[:, 1], s=10)
        plt.plot(0, 0, 'r.', markersize=25)
        plt.title(f"SVD Projection: Condition {condition+1}, Electrode {electrode+1}")
        plt.xlabel("1st Principal Component")
        plt.ylabel("2nd Principal Component")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # Cluster using k-means on waveforms
    km = KMeans(n_clusters=2, random_state=0)
    cluster_labels = km.fit_predict(waveforms.T)  # waveforms.T: shape [n_waveforms x time]

    # Separate clusters
    idx1 = np.where(cluster_labels == 0)[0]
    idx2 = np.where(cluster_labels == 1)[0]

    # Compare total amplitude to identify signal vs noise
    mean1 = np.mean(waveforms[:, idx1], axis=1)
    mean2 = np.mean(waveforms[:, idx2], axis=1)

    amp1 = np.sum(np.abs(mean1))
    amp2 = np.sum(np.abs(mean2))

    if amp1 > amp2:
        signal_idx = idx1
        noise_idx = idx2
    else:
        signal_idx = idx2
        noise_idx = idx1

    # Detailed plotting if verbose
    if verbose:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        ax1.scatter(projections[noise_idx, 0], projections[noise_idx, 1], color='b', s=10, label="Noise")
        ax1.scatter(projections[signal_idx, 0], projections[signal_idx, 1], color='g', s=10, label="Signal")
        ax1.plot(0, 0, 'r.', markersize=25)
        ax1.set_title("SVD Clustering")
        ax1.set_xlabel("1st PC")
        ax1.set_ylabel("2nd PC")
        ax1.legend()
        ax1.grid(True)

        # Plot error bars of signal and noise mean traces
        timepoints = np.arange(waveforms.shape[0])
        signal_waveforms = waveforms[:, signal_idx].T
        noise_waveforms = waveforms[:, noise_idx].T

        ax2.errorbar(
            timepoints,
            np.mean(signal_waveforms, axis=0),
            yerr=np.std(signal_waveforms, axis=0),
            label="Signal", color='g'
        )
        ax2.errorbar(
            timepoints,
            np.mean(noise_waveforms, axis=0),
            yerr=np.std(noise_waveforms, axis=0),
            label="Noise", color='b'
        )
        ax2.set_xlim([0, 49])
        ax2.set_title("Mean Waveform with Std Dev")
        ax2.set_xlabel("Time")
        ax2.set_ylabel("Amplitude")
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()
        plt.show()
    else:
        noise_waveforms = waveforms[:, noise_idx].T

    return signal_idx, np.mean(noise_waveforms, axis=0)



In [6]:
def plot_signal_waveforms_aligned(pepANA_clean, condition, signal_idx, electrode=0, sampling_rate=30000.0):
    """
    Plot each waveform in `signal_idx` aligned to its actual time points.

    Parameters:
        pepANA_clean : dict
            MATLAB structure loaded via scipy.io.loadmat
        condition : int
            Index into the listOfResults (0-based)
        signal_idx : array-like
            Indices of signal-classified waveforms
        electrode : int
            Which electrode (0-based index into .data)
        sampling_rate : float
            Sampling rate in Hz (default 30 kHz)
    """

    arrival_times = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][0]  # shape: (N,)
    waveforms = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1]       # shape: (48, N)

    # Constants
    waveform_length = waveforms.shape[0]                   # 48 samples
    time_step = 1.0 / sampling_rate                        # e.g., 1 / 30000 seconds

    plt.figure(figsize=(12, 8))

    for idx in signal_idx:
        t0 = arrival_times[idx]                            # starting time of spike
        waveform = waveforms[:, idx]
        t_waveform = t0 + np.arange(waveform_length) * time_step  # 48 time samples starting at t0

        plt.plot(t_waveform * 1000, waveform, alpha=0.4)   # convert to ms

    plt.xlabel("Time (ms)")
    plt.ylabel("Amplitude (int8 units)")
    plt.title(f"Signal-Aligned Waveforms (n={len(signal_idx)})")
    plt.grid(True)
    plt.tight_layout()

    plt.show()


In [7]:
def collect_mean_noise_waveforms_and_signal_indices(pepANA_clean, condition=0, verbose=False):
    """This will collect all the mean noise waveforms to be used when a waveform is not detected from the stimulus in another electrode. 
    This will also return all indices of detected signals.

    Args:
        pepANA_clean (dict): This is the dict containing all 120 conditions and meta data for the trials.
        condition (int): This is the condition that is being inspected.
        verbose (bool): If true, the detected mean signal waveform and corresponding mean noise will be plot.
    
    Returns:
        mean_noise_waveforms (np.ndarray): 16 x 48 mean-noise waveforms for a single condition.
        signal_idx_l (list): Each list index is an np.array of indices of detected signals for use 
            in the pepANA_clean["listOfResults"][0]["repeat"]["data"][electrode][1] for signal 
            and pepANA_clean["listOfResults"][0]["repeat"]["data"][electrode][0] for initial starting times.
    """
    mean_noise_waveforms = np.zeros((16,48))
    signal_idx_l = []

    for electrode in range(16):
        signal_indices, mean_noise_waveforms[electrode] = analyze_waveform(pepANA_clean, condition=condition, electrode=electrode, verbose=verbose)
        signal_idx_l.append(signal_indices)
    return signal_idx_l, mean_noise_waveforms


In [8]:
# Calculate the global mean and standard deviation
def calculate_global_mean_and_std(pepANA_clean, write=False, filename="electrode_global_mean_std.json"):
    global_electrode_mean_std = {}
    for electrode in range(16):
        all_signal = pepANA_clean["listOfResults"][0]["repeat"]["data"][electrode][1].flatten()
        for condition in range(1, 120):
            signal = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1].flatten()
            all_signal = np.concatenate((all_signal, signal))
        global_electrode_mean_std[electrode] = {"mean": np.mean(all_signal), "std": np.std(all_signal)}
        del all_signal
    if write:
        with open(filename, "w") as f:
            json.dump(global_electrode_mean_std, f)
    return global_electrode_mean_std

In [9]:
def identify_stimulus_images(pepANA_clean, signal_idx, condition=0, electrode=0, latency=60e-3):
    FRAME_PERIOD = 1/30
    frame_indices = ((pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][0][signal_idx] - latency) // FRAME_PERIOD).astype(int)
    
    # Movie_id is 0, 1, 2, or 3; 
    movie_id, segment_id = pepANA_clean["listOfResults"][condition]["values"]
    movie_id_zeros_n = (3 - len(str(abs(movie_id))))
    movie_id_zeros = repr(0)*movie_id_zeros_n
    movie_id_formatted = movie_id_zeros + str(movie_id)

    # segment_id is greater than 10 less than 100
    segment_id_zeros_n = (3 - len(str(abs(segment_id))))
    segment_id_zeros = repr(0)*segment_id_zeros_n
    segment_id_formatted =  segment_id_zeros + str(segment_id)
    
    # Frame Indices Format
    frame_indices_formatted = [str(idx).zfill(3) for idx in frame_indices]

    movie_frame_str_l = ["movie" + movie_id_formatted + "_" + segment_id_formatted + "_" + frame_indices_formatted_idx + ".jpeg" for frame_indices_formatted_idx in frame_indices_formatted]
    movie_frames_root_path = "../data/crcns-pvc1/crcns-ringach-data/movie_frames/"
    movie_frame_dir = "movie" + movie_id_formatted + "_" + segment_id_formatted + ".images"
    full_image_path_l = [movie_frames_root_path + movie_frame_dir + "/" + movie_frame_str for movie_frame_str in movie_frame_str_l]
    
    return full_image_path_l

### Testing image data load

In [ ]:
path = "../data/crcns-pvc1/crcns-ringach-data/movie_frames/movie000_000.images/movie000_000_000.jpeg"

In [ ]:
image = cv2.imread(path , cv2.IMREAD_COLOR)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image = Image.open(movie_frame_path)
image.show()

### Testing neural data load

In [ ]:
neuro_data_path = os.path.join(data_root_dir, neuro_root_dir, current_neuro_data)
mat = loadmat(neuro_data_path, struct_as_record=False, squeeze_me=True)
pepANA = mat["pepANA"]
pepANA_clean = matlab_to_python(pepANA)

In [ ]:
# Total Number of Conditions
pepANA_clean["no_conditions"]

In [ ]:
# Arrival Times for electrodes in pepANA.electrode_list
pepANA_clean["listOfResults"][0]["repeat"]["data"][12]

In [ ]:
# Detected Waveforms at that time
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1].shape

In [ ]:
# Times since initial waveform is detected. (48 samples of waveform snippets)
# (Contains spikes and noise bc of low threshold 
# (1.6ms sampled at 30kHz at int8 precision)) 

pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0].shape

In [ ]:
# Extract waveform data and spike times
waveforms = pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1]  # (48, 573)
arrival_times = pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0]  # (573,)


In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0]

In [ ]:
# Parameters
n_waveforms = waveforms.shape[1]
n_samples = waveforms.shape[0]

In [ ]:
sampling_rate = 30_000  # Hz
offset = 200  # vertical spacing between waveforms
time_axis = np.arange(n_samples) / sampling_rate * 1_000  # in milliseconds (0–1.6 ms)

In [ ]:
n_waveforms

In [ ]:
waveform_t = (arrival_times[0] + np.arange(n_samples) / 30000)

In [ ]:
n_samples

In [ ]:
arrival_times[0]

In [ ]:
waveforms[:,0]

In [ ]:
waveforms.shape[1]

In [ ]:
t = 0

In [ ]:
t += 1
t = t % waveforms.shape[1]
waveform_t = (arrival_times[t] + np.arange(n_samples) / 30000)
plt.plot(waveform_t, waveforms[:,t])

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1].shape

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][0]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1][:, 0]

In [ ]:
plt.plot()

## Calculate the global mean and standard deviation for each of the 16 electrodes across all samples

In [10]:
neuro_data_path = os.path.join(data_root_dir, neuro_root_dir, current_neuro_data)
mat = loadmat(neuro_data_path, struct_as_record=False, squeeze_me=True)
pepANA = mat["pepANA"]
pepANA_clean = matlab_to_python(pepANA)


In [14]:
global_electrode_mean_std = calculate_global_mean_and_std(pepANA_clean, write=True, filename="electrode_global_mean_std.json")

In [15]:
global_electrode_mean_std

{0: {'mean': -1.7976207342956518, 'std': 18.620448315184728},
 1: {'mean': -2.052581657807565, 'std': 13.941962207500119},
 2: {'mean': -1.855526213592233, 'std': 13.787108913800049},
 3: {'mean': -1.0824869529082486, 'std': 15.445568115641649},
 4: {'mean': -1.1331206276667172, 'std': 25.058158919364697},
 5: {'mean': -0.49429660614690873, 'std': 24.479246493596893},
 6: {'mean': -2.0114783511832424, 'std': 9.379316165769762},
 7: {'mean': -0.9960047160637884, 'std': 25.411313235799682},
 8: {'mean': -2.0144884966391605, 'std': 24.848789813699597},
 9: {'mean': -1.5876223494190826, 'std': 14.263069858221387},
 10: {'mean': -1.7129196660667867, 'std': 14.03654235360675},
 11: {'mean': -2.0415448419010356, 'std': 12.33214201132105},
 12: {'mean': -3.1126146195089257, 'std': 14.972900862614798},
 13: {'mean': -1.1270642555199275, 'std': 15.712746337360366},
 14: {'mean': -1.3769917103530203, 'std': 18.881028669358148},
 15: {'mean': -1.1975715679541268, 'std': 16.410864896346347}}

## SVD of Waveforms

In [ ]:
neuro_data_path = os.path.join(data_root_dir, neuro_root_dir, current_neuro_data)
mat = loadmat(neuro_data_path, struct_as_record=False, squeeze_me=True)
pepANA = mat["pepANA"]
pepANA_clean = matlab_to_python(pepANA)
signal_idx, mean_noise_waveform = analyze_waveform(pepANA_clean, 0, verbose=True)

In [ ]:
neuro_data_path

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1].shape

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0].shape

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][:5]

In [ ]:
signal_idx, mean_noise_waveform = analyze_waveform(pepANA_clean, 0, verbose=True)

In [ ]:
signal_idx.__len__()

In [ ]:
# Arrival Times of Detected Signals
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][signal_idx]

In [ ]:
# detected_signal_times_for_ch_0 = pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][signal_idx]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][signal_idx]

In [ ]:
# Waveforms that are 48 samples long that are verifiably signal
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1][:, signal_idx].shape

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][1][1].shape

# pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1][:, signal_idx]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][1][1].shape

In [ ]:
sample_time = pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][signal_idx][0]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0][signal_idx]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][1][0]

In [ ]:
plot_signal_waveforms_aligned(pepANA_clean, 0, signal_idx, electrode=0, sampling_rate=30000.0)

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"]

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"]

In [ ]:
condition = 0
electrode +=1 
electrode %= 16

pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1].shape

In [ ]:
# Associating frames with spike signals
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][1][:, signal_idx]

In [ ]:
movie_id, segment_id = pepANA_clean["listOfResults"][0]["values"]
movie_id, segment_id
signal_idx

In [ ]:
# Time between frames and detected signals
latency = 60e-3
frame_period = 1/30 # Hz

In [ ]:
pepANA_clean["listOfResults"][0]["repeat"]["data"][0][0]

## Identifying the stimulus image

In [ ]:
full_image_path_l = identify_stimulus_images(pepANA_clean, signal_idx, condition=0, electrode=0, latency=60e-3)

In [ ]:
full_image_path_l.__len__()

In [ ]:
full_image_path = full_image_path_l[0]

In [ ]:
image = cv2.imread(full_image_path, cv2.IMREAD_COLOR)
cv2.imshow("Image", image) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

Next Steps:
- [x] I identify all 16x48 waveforms that are associated with spiking activity of each of the 16 electrodes for all 120 conditions. 
- [x] I use SVD to detect the signal indices, 
- [ ] collect the movie images, 
- [x] identify the other 15 electrode waveforms at the spike time of the original electrode, 
- [x] normalize these waveforms with a per-channel zero mean and unit variance. Then I 
- [ ] define the architecture of a neural network that will generate an image based upon a 16x48 waveform input. Then I 
- [ ] iterate through the 16x48 waveforms that I have and 
- [ ] Create a Train and test dataset for the neural network
- [ ] Train the neural network
- [ ] iterate through the 16x48 waveforms that I have randomly
- [ ] send the 16x48 waveforms through a websocket endpoint to a relay 
- [ ] generate an image in the relay using the trained neural network
- [ ] Visualize the image in a react frontend.


## identify all 16x48 waveforms that are associated with spiking activity of each of the 16 electrodes for all 120 conditions. 

I need to preallocate an array that is Nx16x48 where N is the number of detected spikes. Then for each channel I need to identify if there is a matching spike on another channel by using the detected spike times of the original channel to search the other arrays. If there is a detected starting time then I need to extract the 48 samples from that starting time and populate the array with those samples at the corresponding signal in the initial detected electrode. All non-detected times will be set to zero so the pre-allocation must be initialized to zero. 

Then I will iterate to the next electrode for all 16 electrodes and use SVD to detect spike times and the corresponding stimulus images list. Then I will perform the same function as defined above:

I need to preallocate an array that is Nx16x48 where N is the number of detected spikes. Then for each channel I need to identify if there is a matching spike on another channel by using the detected spike times of the original channel to search the other arrays. If there is a detected starting time then I need to extract the 48 samples from that starting time and populate the array with those samples at the corresponding signal in the initial detected electrode. All non-detected times will be set to zero so the pre-allocation must be initialized to zero. 

Then I will move to the next condition and perform this operation for all 120 conditions identifying all Nx16x48 waveform samples and N stimulus images for 16 electrodes for all 120 conditions. I will save the 16xNx16x48 Numpy array and the 16xN list of images for all 120 electrodes where N is the number of detected signals and corresponding stimulus images. 

Each 16x48 sample is only representative of a single electrode that is associated with the image. I have N samples for the single electrode for all 16 electrodes. I need to train the model such that the features that each electrode responds to are captured by the model such that when I have 16 arbitrary waveforms of 48 samples long, the resulting image is correspondent to the features that stimulated those electrodes initially. I do not want to train the model such that there are 1 stimulus electrode and 15 noise or zero value electrodes and then send a sample that only generates the single feature. I want to send a signal that comprises all electrodes and generates the corresponding image as a result. The other channels may be noise, but I dont want to train the model to detect only a single channel. I want the model to capture the features that each channel corresponds to in the stimulus image. 

Use SVD on the other channels. Capture the mean noise signal. This will be used as a placeholder for signals that are non-present.


In [16]:
neuro_data_path = os.path.join(data_root_dir, neuro_root_dir, current_neuro_data)
mat = loadmat(neuro_data_path, struct_as_record=False, squeeze_me=True)
pepANA = mat["pepANA"]
pepANA_clean = matlab_to_python(pepANA)

In [ ]:
signal_idx_l, mean_noise_waveforms = collect_mean_noise_waveforms_and_signal_indices(pepANA_clean, condition=condition, verbose=False)

In [19]:
# Important
LATENCY = 60e-3
# for each electrode, identify all the stimulus spikes and the waveforms for the other 15 electrodes at the stimulus spike times. If the waveforms are not found, use the mean noise waveform.
# Initialize nested dictionary
collected_stimulus_images_from_all_electrodes_for_all_conditions_dict = defaultdict(lambda: defaultdict(list))
collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l = []

for condition in tqdm(range(120), desc="Preprocessing conditions", ascii="░▒▓█"):
    signal_idx_l, mean_noise_waveforms = collect_mean_noise_waveforms_and_signal_indices(pepANA_clean, condition=condition, verbose=False)
    
    # COLLECT ALL WAVEFORMS FOR THE 16 ELECTRODES FOR THE CURRENT CONDITION AND NORMALIZE PER ELECTRODE
    collected_stimulus_waveforms_from_all_electrodes_l = []
    for init_electrode in range(16):
        spike_times_to_id = pepANA_clean["listOfResults"][condition]["repeat"]["data"][init_electrode][0][signal_idx_l[init_electrode]]
        
        # COLLECT THE STIMULUS IMAGES 
        collected_stimulus_images_from_all_electrodes_for_all_conditions_dict[condition][init_electrode] = identify_stimulus_images(pepANA_clean, signal_idx=signal_idx_l[init_electrode], condition=condition, electrode=init_electrode, latency=LATENCY)        
        
        
        collected_stimulus_waveforms_from_all_electrodes = np.zeros((len(spike_times_to_id), 16, 48))
        time_idx = 0
        for time in spike_times_to_id:
            for electrode in range(16):
                if electrode == init_electrode:
                    for time in spike_times_to_id:
                        # find index of 'time' in the time array of electrode "electrode"
                        time_arr = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][0]
                        # find index where time matches exactly
                        time_indices = np.where(time_arr == time)[0]

                        if time_indices.size > 0:
                            time_index = time_indices[0]  # take first match
                            waveform = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1][:, time_index].flatten()
                            # normalize waveform to zero mean and unit variance using the global statistics per electrode
                            waveform = (waveform - global_electrode_mean_std[electrode]["mean"]) / global_electrode_mean_std[electrode]["std"]
                            collected_stimulus_waveforms_from_all_electrodes[time_idx][electrode] = waveform
                

                time_arr = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][0]
                temporal_index = np.nonzero(np.isin(time_arr, time))[0]
                if len(temporal_index) > 0:
                    waveform = pepANA_clean["listOfResults"][condition]["repeat"]["data"][electrode][1][:, temporal_index].flatten()
                    # normalize waveform to zero mean and unit variance using the global statistics per electrode
                    waveform = (waveform - global_electrode_mean_std[electrode]["mean"]) / global_electrode_mean_std[electrode]["std"]
                    collected_stimulus_waveforms_from_all_electrodes[time_idx][electrode] = waveform
                else:
                    # normalize waveform to zero mean and unit variance using the global statistics per electrode
                    collected_stimulus_waveforms_from_all_electrodes[time_idx][electrode] = (mean_noise_waveforms[electrode] - global_electrode_mean_std[electrode]["mean"]) / global_electrode_mean_std[electrode]["std"]
            time_idx +=1 
        collected_stimulus_waveforms_from_all_electrodes_l.append(collected_stimulus_waveforms_from_all_electrodes)
    collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l.append(collected_stimulus_waveforms_from_all_electrodes_l)

Preprocessing conditions: 100%|██████████| 120/120 [10:02<00:00,  5.02s/it]


In [21]:
collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l.__len__()

120

In [23]:
collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l[0].__len__()

16

In [25]:
collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l[0][0].__len__()

192

In [27]:
# Test that each detected stimulus signal are associated with a stimulus image
for condition in range(120):
    for electrode in range(16):
        assert collected_stimulus_images_from_all_electrodes_for_all_conditions_dict[condition][electrode].__len__() == collected_stimulus_waveforms_from_all_electrodes_l_for_all_conditions_l[condition][electrode].__len__()